In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "3_clean")

In [2]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3


## Load building level analysis file

In [12]:
df_bbl = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
df_bbl.GetContentFile('analysis_bbl.csv')  

df_bbl= pd.read_csv('analysis_bbl.csv').drop(["Unnamed: 0"], axis = 1)
df_bbl.head()

df_bbl.drop(df_bbl.loc[df_bbl["bbl"].isna()].index, inplace = True)

C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Decision Trees and Random Forest Models

In [20]:
assert df_bbl.loc[:, "vacate_ind"].isna().sum() == 0
df_bbl.loc[:, "num_vac_orders"]= df_bbl.loc[:, "num_vac_orders"].fillna(0)

In [7]:
print(df_bbl.columns)
df_bbl.describe()

Index(['borough', 'bbl', 'cd', 'tract2010', 'zipcode', 'address', 'bldgclass',
       'landuse', 'ownertype', 'ownername',
       ...
       'pctnhasian', 'pctnhhawainpi', 'pctnhother', 'pctnhtwoplus',
       'pctownerocch', 'pctrentocch', 'crowding', 'crowdingrenter',
       'medianhhincome_imp', 'geofips_recode'],
      dtype='object', length=157)


,bbl,cd,tract2010,zipcode,landuse,lotarea,bldgarea,numbldgs,numfloors,unitsres,...,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,medianhhincome_imp,geofips_recode
count,7.650280e+05,765028.000000,765023.000000,765028.000000,765028.000000,7.650280e+05,7.650280e+05,765028.000000,765028.000000,765028.000000,...,765019.000000,765019.000000,765019.000000,765019.000000,765008.000000,765008.000000,765008.000000,764994.000000,765080.000000,7.650800e+05
mean,3.548738e+09,357.695427,9565.767255,11026.317894,1.421476,3.812045e+03,5.051024e+03,1.324974,2.443931,4.689660,...,15.733118,0.033563,1.119466,1.962125,50.197490,49.802510,8.170169,10.205403,0.001171,3.541736e+06
std,1.003516e+09,98.074679,26807.879379,464.453474,0.842763,2.318735e+04,3.520150e+04,2.726637,1.676084,42.058941,...,17.564558,0.269547,3.038153,2.060769,24.959841,24.959841,6.977935,9.219096,0.034202,9.994781e+05
min,1.000070e+09,0.000000,1.000000,0.000000,1.000000,2.300000e+01,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000100e+06
25%,3.036730e+09,307.000000,222.000000,10467.000000,1.000000,2.000000e+03,1.440000e+03,1.000000,2.000000,1.000000,...,2.887910,0.000000,0.000000,0.490998,29.103943,27.593699,3.289474,3.183024,0.000000,3.030100e+06
50%,4.012680e+09,403.000000,504.000000,11223.000000,1.000000,2.500000e+03,2.020000e+03,1.000000,2.000000,2.000000,...,8.355388,0.000000,0.000000,1.404125,50.627615,49.372385,6.207928,8.142494,0.000000,4.010200e+06
75%,4.107290e+09,412.000000,1139.000000,11372.000000,2.000000,4.000000e+03,2.923000e+03,2.000000,2.750000,2.000000,...,23.870968,0.000000,0.778210,2.694064,72.406301,70.896057,11.000000,14.855072,0.000000,4.092200e+06
max,5.080500e+09,503.000000,157903.000000,12345.000000,4.000000,1.225343e+07,1.354011e+07,1807.000000,98.000000,19201.000000,...,91.605249,7.191448,31.090573,20.276498,100.000000,100.000000,100.000000,100.000000,1.000000,5.990100e+06


In [8]:
df_bbl= pd.get_dummies(df_bbl, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [21]:
# define predictors
df_bbl.columns
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi"]
predictors_acs= df_bbl.columns[df_bbl.columns.get_loc("totpop"):df_box.columns.get_loc("crowdingrenter") + 1].tolist()
predictors_full= predictors + predictors_acs
predictors_full

['lotarea',
 'bldgarea',
 'numbldgs',
 'numfloors',
 'unitsres',
 'unitstotal',
 'assessland',
 'assesstot',
 'exempttot',
 'yearbuilt',
 'holc_AB',
 'holc_CD',
 'holc_D',
 'tot_hpd_vio',
 'hpd_fire_vio',
 'hpd_b_c_vio',
 'viol_count_per_unit',
 'hpd_fire_vio_per_unit',
 'hpd_b_c_vio_per_unit',
 'AL',
 'NB',
 'average_energy_usage',
 'average_energy_usage_mi']

In [29]:
outcomes= ["vacate_ind", "num_vac_orders"]
df_bbl[outcomes].head()

# very small percentage of the data has had a fire
df_bbl[outcomes[0]].mean()

0.0034037969852083847

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X = df_bbl.loc[:, predictors_full].fillna(0)
y = df_bbl.loc[:, "vacate_ind"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

dt = DecisionTreeClassifier(max_depth=10)
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
accuracy= (y_predict == y_test).value_counts(normalize=True)[True]
accuracy

0.9962441560025969

__Area under the curve__

In [36]:
from sklearn.metrics import roc_auc_score
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
# predict_proba predicts the probability of each class rather than just the most likely class
pred=dt.predict_proba(X_test)[:,1] # predicted probability of y = 1
roc_auc_score(np.array(y_test),pred)

0.5641451299836926

In [37]:
# vs. random chance (only slightly better)
pred=np.random.random(len(X_test))
roc_auc_score(np.array(y_test.apply(int)),pred)

0.4883001331387622

## Tuning

In [38]:
from sklearn.model_selection import GridSearchCV
# This time we'll use max_depth to control the complexity of the tree, still using the same train/test split as above,
# and optimize the parameter value using GridSearchCV.
param_grid = {'max_depth':range(1,20)}
dt=DecisionTreeClassifier()
gr=GridSearchCV(dt,param_grid=param_grid,scoring='roc_auc')
rs=gr.fit(X_train,y_train)
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),rs.predict_proba(X_test)[:,1]))

{'max_depth': 6}
0.9736540027162683


## Feature Importance

In [39]:
dt = DecisionTreeClassifier(max_depth=rs.best_params_['max_depth'])
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 5 most important features in order
Feature_importance.sort_values(by="importance",ascending=False).iloc[:5,:]

,variables,importance
15,hpd_b_c_vio,0.526278
1,bldgarea,0.221705
13,tot_hpd_vio,0.099934
7,assesstot,0.027771
5,unitstotal,0.027711


## Random Forests

In [40]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_leaf_nodes=10)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test,pred))

0.975417745523296


In [41]:
param_grid = {'n_estimators':range(1,51,5)}
rf = RandomForestClassifier(n_jobs=-1,max_leaf_nodes=10)
gs = GridSearchCV(rf,param_grid=param_grid,scoring='roc_auc')
rs = gs.fit(X_train,y_train)
pred=rs.predict_proba(X_test)[:,1]
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),pred))

{'n_estimators': 41}
0.9741796240253671
